# Kernel Methods challenge

Importing base libraries...

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

## Debugging requirements
import pdb

## Performance metrics requirements
import time

## Kernel SVM requirements
from cvxopt import matrix
from cvxopt import solvers
import mosek

from scipy.spatial.distance import cdist
from numpy.core.defchararray import not_equal

## 1.Loading the data + sanity checks

In [2]:
%run data_handler.py

## Loading training data
tr0 = load_data(0, 'tr')
tr1 = load_data(1, 'tr')
tr2 = load_data(2, 'tr')

## Loading test data
te0 = load_data(0, 'te')
te1 = load_data(1, 'te')
te2 = load_data(2, 'te')

Some sanity checks...

<b>Training set 0</b>

In [ ]:
tr0['Bound'].describe()

In [ ]:
tr0.head(5)

In [ ]:
tr0.tail(5)

<b>Training set 1</b>

In [ ]:
tr1['Bound'].describe()

In [ ]:
tr1.head(5)

In [ ]:
tr1.tail(5)

<b>Training set 2</b>

In [ ]:
tr2['Bound'].describe()

In [ ]:
tr2.head(5)

In [ ]:
tr2.tail(5)

<b>Test set 0</b>

In [ ]:
te0['Sequence'].describe()

In [ ]:
te0.head(5)

In [ ]:
te0.tail(5)

<b>Test set 1</b>

In [ ]:
te1['Sequence'].describe()

In [ ]:
te1.head(5)

In [ ]:
te1.tail(5)

<b>Test set 2</b>

In [ ]:
te2['Sequence'].describe()

In [ ]:
te2.head(5)

In [ ]:
te2.tail(5)

First idea: use some distance on the strings as a kernel.
However, note that some distances (Hamming) are only defined for sequences of the same size.
What is the mininimum and maximum length of the DNA sequences in this first train set?

In [ ]:
min_length = tr0['Sequence'].str.len().max(0)
max_length = tr0['Sequence'].str.len().max(0)
print('Min sequence length: {}'.format(min_length))
print('Max sequence length: {}'.format(max_length))
print('Length amplitude: {}'.format(max_length-min_length))

## 2. Defining first kernels + running simple classification model

### First kernels

Ok, so here all sequences have the same length. That means that we can start by something simple like Hamming. However, we may want to use something that would seamlessly extend to DNA sequences of different lengths...
Here I will test both the Hamming and the Levenshtein distance as kernels for mapping DNA sequences:

In [3]:
%run kernels.py

Testing kernel computation speed (debugging only):

In [ ]:
t0 = time.time()
Ktr0 = build_kernel(tr0['Sequence'], tr0['Sequence'], kernel_fct = hamming_distance)
t1 = time.time()
Ktr1 = build_kernel(tr1['Sequence'], tr1['Sequence'], kernel_fct = hamming_distance)
t2 = time.time()
Ktr2 = build_kernel(tr2['Sequence'], tr2['Sequence'], kernel_fct = hamming_distance)
t3 = time.time()

In [ ]:
print('Preparing kernel matrix for a training dataset 1 took {0:d}min {1:d}s with this method'.format(int((t1-t0)/60),int(t1-t0)%60))
print('Preparing kernel matrix for a training dataset 2 took {0:d}min {1:d}s with this method'.format(int((t2-t1)/60),int(t2-t1)%60))
print('Preparing kernel matrix for a training dataset 3 took {0:d}min {1:d}s with this method'.format(int((t3-t2)/60),int(t3-t2)%60))

### Tools

Defining a couple of losses functions that will be useful:

In [4]:
%run metrics.py
        
m_binary = Metric('Match rate', lambda preds,labels: 1 - ls_binary(preds,labels))

### Kernel method parent & kernel SVM

Throughout the challenge we will need to use different kernel methods, which will share some attributes and methods. I will thus create an "abstract" class kernelMethod, and derive a kernelSVM class from it.

The first thing I will try out is a kernel SVM method:

In [23]:
%run kernel_methods.py

## Testing SVM implementation with a linear SVM on iris dataset

First let's test the KernelSVM class that we've built on simple data, coming from the IRIS dataset...

In [ ]:
iris_file  = 'misc_data/Iris.csv'
iris = pd.read_csv(iris_file)

In [ ]:
iris = iris.assign(label=(iris['Species']=='Iris-setosa'))
_ = iris.pop('Species')

In [ ]:
lbda2 = 0.5
lSVM = kernelSVM(lbda2)
iris_X = iris.drop(['Id','label'], axis=1).as_matrix()
iris_Y = iris['label'].as_matrix()

First let's test training a linear SVM on the whole dataset

In [ ]:
lSVM.train(iris_X, iris_Y, kernel_fct=None, stringsData=False)

In [ ]:
iris_preds = np.ravel(format_preds(lSVM.predict(iris_X, stringsData=False)))

Then let's try a cross-validation with 5 folds:

In [ ]:
lSVM.assess(iris_X, iris_Y, n_folds=5, stringsData=False)

In [ ]:
iris_preds.astype(bool)

Great! It looks like the kernelSVM class is fully functionnal on a linear kernel with vector data.

## KernelSVM for predicting transcription factor binding
Now let's try our kernelSVM with some basic kernels:
- based on Hamming distance (acceptable in terms of computation time for our purpose)
- based on Levenshtein distance? (would seem more relevant to the problem, however computational issues are abound)

In [ ]:
## Method definition
lbda = 0.005
kSVM = kernelSVM(lbda)

In [ ]:
## Training SVM + performance assessment on training data
kSVM_tr0_score = kSVM.assess(tr0['Sequence'].as_matrix(), tr0['Bound'].as_matrix(), kernel_fct = hamming_kernel, n_folds = 5, metric=m_binary)
kSVM.train(tr0['Sequence'].as_matrix(), tr0['Bound'].as_matrix(), hamming_kernel)
kSVM_te0_raw = np.sign(kSVM.predict(te0['Sequence'])).astype(int)

In [ ]:
## Training SVM + performance assessment on training data
kSVM.train(tr1['Sequence'], tr1['Bound'], hamming_kernel)
preds_kSVM_tr1, perf_kSVM_tr1 = kSVM.assess(tr1['Sequence'], tr1['Bound'], metrics=[m_binary])
print('Training dataset {0:d}: {1:s}: {2:.1f}%'.format(1, list(perf_kSVM_tr1.keys())[0], 100*list(perf_kSVM_tr1.values())[0]))
kSVM_te1_raw = np.sign(kSVM.predict(te1['Sequence'])).astype(int)

In [ ]:
## Training SVM + performance assessment on training data
kSVM.train(tr2['Sequence'], tr2['Bound'], hamming_kernel)
preds_kSVM_tr2, perf_kSVM_tr2 = kSVM.assess(tr2['Sequence'], tr2['Bound'], metrics=[m_binary])
print('Training dataset {0:d}: {1:s}: {2:.1f}%'.format(2, list(perf_kSVM_tr2.keys())[0], 100*list(perf_kSVM_tr2.values())[0]))
kSVM_te2_raw = np.sign(kSVM.predict(te2['Sequence'])).astype(int)

<b>Current performance rate</b>: ~65% on training set, ?? on test set

<b>Next steps - Results</b>:
- What is the reason for such a poor performance rate, even on the training data?
- If this is due to Hamming being mostly irrelevant, implement the Levenshtein distance and retry with this new kernel

<b>Next steps - Computing speed</b>:
- Find a way to vectorize the kernel matrix computation

In [ ]:
## Predictions on test data
kSVM_te0 = pd.DataFrame(
    data = format_preds(kSVM_te0_raw),
    columns = ['Bound'])

kSVM_te1 = pd.DataFrame(
    data = format_preds(kSVM_te1_raw),
    columns = ['Bound'])
kSVM_te1.index = kSVM_te1.index + 1000

kSVM_te2 = pd.DataFrame(
    data = format_preds(kSVM_te2_raw),
    columns = ['Bound'])
kSVM_te2.index = kSVM_te2.index + 2000

frames = [kSVM_te0, kSVM_te1, kSVM_te2]
kSVM_te = pd.concat(frames)
kSVM_te.index = kSVM_te.index.set_names(['Id'])

kSVM_te.to_csv('predictions/kSVM_te.csv')

### Trying out kNN with Hamming (debugging only)

In [24]:
k = 10
kNN = kernelKNN(k)
kNN.train(tr0['Sequence'], tr0['Bound'])
kNN.predict(te0['Sequence'], hamming_kernel)

/home/proprietaire/Documents/Workspace/School/MVA/Kernel Methods/Kernel_Challenge/kernel_methods.py:31: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  arr1 = arr1.reshape((len(arr1),1))
/home/proprietaire/Documents/Workspace/School/MVA/Kernel Methods/Kernel_Challenge/kernel_methods.py:33: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  arr2 = arr2.reshape((len(arr2),1))


Building kernel matrix from 1000x2000 samples...
...done in 33.66s


array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0,